In [0]:
from pyspark.sql.types import *

In [0]:
spark.sql("""
        CREATE TABLE Dim_Rider
        USING DELTA LOCATION '/delta/bronze_riders'
        """)

Out[2]: DataFrame[]

In [0]:
gold_dim_rider = spark.sql("""
        SELECT rider_id,
               first, 
               last, 
               address,
               birthday,
               account_start_date,
               account_end_date,
               CASE is_member WHEN 'True' THEN 'Member'
                     ELSE 'Casual Rider'
                     END AS member_status
               FROM riders
        """)

In [0]:
gold_dim_rider.show()

+--------+--------+---------+--------------------+----------+------------------+----------------+-------------+
rider_id| first| last| address| birthday|account_start_date|account_end_date|member_status|
+--------+--------+---------+--------------------+----------+------------------+----------------+-------------+
 1000|Kimberly| Williams| 1200 Alyssa Squares|1988-03-28| 2019-04-23| null| Member|
 1001| Anthony| Erickson| 397 Diana Ferry|1976-12-04| 2019-11-01| 2020-09-01| Member|
 1002| Jessica| Roach|644 Brittany Row ...|1998-03-28| 2022-02-04| null| Member|
 1003| Andrew| Ryan|996 Dickerson Tur...|1999-03-05| 2019-08-26| null| Casual Rider|
 1004| Ian| Peters|7009 Nathan Expre...|1969-06-25| 2019-09-14| null| Member|
 1005| Michael|Gillespie|224 Washington Mi...|1974-09-28| 2020-03-24| null| Casual Rider|
 1006| Ryan| Peters| 1137 Angela Locks|2003-07-10| 2020-11-27| 2021-06-01| Member|
 1007| Crystal| Sanchez| 979 Phillips Ways|1987-10-15| 2016-12-11| null| Casual Rider|
 1008| David| Hicks| 7691 Evans Court|1986-07-12| 2021-03-28| 2021-04-01| Member|
 1009| Daniel| Hicks|9922 Jim Crest Ap...|1981-02-14| 2020-06-12| 2021-02-01| Member|
 1010| Ashley| Webster| 92973 Mary Ville|1996-10-31| 2019-12-27| null| Member|
 1011| Rachel| Mills|950 Grimes Burg A...|1984-08-09| 2017-05-20| null| Member|
 1012| Mark| Day|65532 Davis Sprin...|1980-05-06| 2021-10-16| null| Member|
 1013| Corey| Stafford| 7757 Johnston Roads|1986-01-29| 2020-12-28| 2021-08-01| Member|
 1014| Debra| Wilson| 501 Arellano Land|1989-09-14| 2017-11-24| null| Member|
 1015| Kristin| Hunt|3710 Rodriguez Gl...|2001-08-23| 2017-07-10| null| Member|
 1016| Misty| Scott| 72226 Casey Square|1991-10-30| 2022-02-02| null| Member|
 1017| Jason|Mcpherson| 40395 Terrell Parks|1982-01-29| 2019-03-11| null| Member|
 1018| Brandon| Hall| 3514 Leslie Vista|1986-12-15| 2021-07-25| null| Casual Rider|
 1019| David| Thompson|00348 Brandi Park...|1997-07-01| 2021-07-10| 2021-08-01| Casual Rider|
+--------+--------+---------+--------------------+----------+------------------+----------------+-------------+
only showing top 20 rows

In [0]:
gold_dim_rider.write \
  .format("DELTA") \
  .mode("overwrite") \
  .saveAsTable('dim_rider')

In [0]:
spark.sql("""
        CREATE TABLE dim_stations
        USING DELTA LOCATION '/delta/bronze_stations'
""")

Out[17]: DataFrame[]

In [0]:
gold_dim_station = spark.sql("""
                          SELECT station_id,
                          name,
                          latitude,
                          longitude
                          FROM stations
                        """)

In [0]:
gold_dim_station.write \
  .format("DELTA") \
  .mode("overwrite") \
  .saveAsTable('dim_stations')

In [0]:

spark.sql("""
        CREATE TABLE fact_trips
        USING DELTA LOCATION '/delta/bronze_trips'
        """)

Out[28]: DataFrame[]

In [0]:
spark.sql("""
        CREATE TABLE fact_payments
        USING DELTA LOCATION '/delta/bronze_payments'
         """)

Out[29]: DataFrame[]

In [0]:
gold_fact_payment = spark.sql("""
         SELECT 
            fact_payments.payment_id,
            bigint(date_format(fact_payments.date, 'yyyyMMddHH')) as date,
            dim_rider.rider_id aS rider_id,
            fact_payments.amount as amount
            FROM fact_payments
            LEFT JOIN dim_rider
            ON fact_payments.rider_id = dim_rider.rider_id;
         """)

In [0]:
gold_fact_payment.write \
  .format("DELTA") \
  .mode("overwrite") \
  .saveAsTable('fact_payment')

In [0]:
gold_fact_trips = spark.sql("""
          SELECT 
            CONCAT(fact_trips.trip_id, CAST(dim_rider.rider_id AS varchar(10))) AS rideable_type,
            fact_trips.start_at as starts_at,
            fact_trips.ended_at as ends_at,
            fact_trips.start_station_id AS station_id,
            fact_trips.end_station_id AS end_station_id,
            dim_rider.rider_id AS rider_id,
            INT(months_between(dim_rider.account_start_date, dim_rider.birthday) / 12) AS rider_age,
            BIGINT(fact_trips.ended_at) - BIGINT(fact_trips.start_at)  AS trip_duration
            FROM fact_trips
            LEFT JOIN dim_rider
            ON fact_trips.rider_id = dim_rider.rider_id;
         """)

In [0]:
gold_fact_trips.write \
  .format("DELTA") \
  .mode("overwrite") \
  .saveAsTable('fact_trip')

In [0]:
beginDate = '2000-01-31'
endDate = '2022-09-01'

(
  spark.sql(f"select explode(sequence(to_timestamp('{beginDate}'), to_timestamp('{endDate}'), interval 1 hour)) as calendarDateTime")
    .createOrReplaceTempView('datetimes')
)


In [0]:
gold_dim_dates = spark.sql("""
     SELECT bigint(date_format(calendarDateTime, 'yyyyMMddHH')) AS date,
     hour(calendarDateTime) AS hour,
     dayofweek(calendarDateTime) AS day_of_week,
     dayofmonth(calendarDateTime) AS day_of_month,
     extract(week FROM calendarDateTime) AS week_of_year,
     date_format(calendarDateTime, 'Q') AS quarter,
     extract(month FROM calendarDateTime) AS month,
     extract(year FROM calendarDateTime) AS year
    FROM datetimes
""")

In [0]:
gold_dim_dates.show()

+----------+----+-----------+------------+------------+-------+-----+----+
 date|hour|day_of_week|day_of_month|week_of_year|quarter|month|year|
+----------+----+-----------+------------+------------+-------+-----+----+
2000013100| 0| 2| 31| 5| 1| 1|2000|
2000013101| 1| 2| 31| 5| 1| 1|2000|
2000013102| 2| 2| 31| 5| 1| 1|2000|
2000013103| 3| 2| 31| 5| 1| 1|2000|
2000013104| 4| 2| 31| 5| 1| 1|2000|
2000013105| 5| 2| 31| 5| 1| 1|2000|
2000013106| 6| 2| 31| 5| 1| 1|2000|
2000013107| 7| 2| 31| 5| 1| 1|2000|
2000013108| 8| 2| 31| 5| 1| 1|2000|
2000013109| 9| 2| 31| 5| 1| 1|2000|
2000013110| 10| 2| 31| 5| 1| 1|2000|
2000013111| 11| 2| 31| 5| 1| 1|2000|
2000013112| 12| 2| 31| 5| 1| 1|2000|
2000013113| 13| 2| 31| 5| 1| 1|2000|
2000013114| 14| 2| 31| 5| 1| 1|2000|
2000013115| 15| 2| 31| 5| 1| 1|2000|
2000013116| 16| 2| 31| 5| 1| 1|2000|
2000013117| 17| 2| 31| 5| 1| 1|2000|
2000013118| 18| 2| 31| 5| 1| 1|2000|
2000013119| 19| 2| 31| 5| 1| 1|2000|
+----------+----+-----------+------------+------------+-------+-----+----+
only showing top 20 rows

In [0]:
gold_dates.write \
  .format("DELTA") \
  .mode("overwrite") \
  .saveAsTable('dim_dates')